# Using Solar Docvision Preview on SageMaker JumpStart




**Solar Docvision Preview** is a preview of our upcoming model, Solar DocVision. Solar DocVision Preview supports information extraction through two main approaches: Extractive QA and Key Information Extraction. Extractive QA focuses on extracting relevant information from documents to answer specific questions. Key Information Extraction, while similar, specializes in extracting structured data, often in formats like JSON. For example, it might extract data such as `{"name": "Jameson", "phone": "xx-xxx-xxx"}`. Both approaches involve analyzing documents to retrieve valuable information, but they differ in their output format and typical use cases.

This sample notebook shows you how to deploy [Solar Docvision Preview](https://aws.amazon.com/marketplace/pp/prodview-astvoyl5bqpfi) using Amazon SageMaker.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Prepare input payload](#B.-Prepare-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
3. [Clean-up](#4.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   2. [Delete the model](#B.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open [Solar Docvision Preview](https://aws.amazon.com/marketplace/pp/prodview-astvoyl5bqpfi) model package listing page.
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

We offer one type of package, `ml.g5.12xlarge`

In [ ]:
%pip install sseclient-py

In [ ]:
import time
import json

import sseclient

import boto3
import sagemaker
from sagemaker import ModelPackage, get_execution_role

In [ ]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()

sagemaker_runtime = boto3.client("sagemaker-runtime")

In [ ]:
model_package_name = "solar-docvision-preview-r1-62c0ab062fb431a5afed104b4da9158c"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{model_package_name}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{model_package_name}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{model_package_name}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{model_package_name}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{model_package_name}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{model_package_name}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{model_package_name}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{model_package_name}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{model_package_name}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{model_package_name}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{model_package_name}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{model_package_name}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{model_package_name}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{model_package_name}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{model_package_name}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{model_package_name}",
}

region = sagemaker_session.boto_region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]

print(f"Model Package: '{model_package_arn}'")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name = "solar-docvision"
content_type = "application/json"

real_time_inference_instance_type = "ml.g5.12xlarge"

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

endpoint_name = sagemaker.utils.name_from_base(model_name)
print(f"endpoint name: '{endpoint_name}'")

In [ ]:
# Deploy the model
model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Prepare input payload

We support request/response payloads in the style of OpenAI's Chat completion endpoint, but with an additional required image file, which can be provided either in base64 format or as a web URL.

Supported parameters:
- messages(list of objects)*: List of messages that contains `role` and `content`. `role` must be `user`.
  - content (list of objects): The content of the user message, which can be of two types:
    - type (string): The type of content, either `text` or `image_url`.
    - text (string): The text content. This is required only if the type is `text`.
    - image_url (string): The URL of the image content. This field is required only if the type is `image_url`. The image can be in base64 format or a web URL.
- model(string): You must specify `model` parameter. Since we have only one model, this parameter is always `solar-docvision`.
- max_tokens: The maximum number of tokens that can be generated in the chat completion. Solar DocVision Preview supports maximum 8k(8192) context for input and generated tokens.
- temperature: What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
- top_p: An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.

**required*

In [ ]:
input = {
    "model": "solar-docvision",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/ReceiptSwiss.jpg/340px-ReceiptSwiss.jpg"
                    },
                },
                {"type": "text", "text": "How much is Latte Macchiato?"},
            ],
        }
    ],
    "max_tokens": 100,
}

### C. Perform real-time inference

In [ ]:
# real-time inference
def invoke_endpoint(endpoint_name, payload):
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
    )
    data = response["Body"].read().decode("utf-8")

    return data

In [ ]:
response = json.loads(invoke_endpoint(endpoint_name, input))

In [ ]:
print(response)

In [ ]:
print("assistant: ")
print(response["choices"][0]["message"]["content"])

### D. Perform real-time inference with Streaming

In [ ]:
input["stream"] = True

In [ ]:
def invoke_endpoint_stream(endpoint_name, request_body):
    response = sagemaker_runtime.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name,
        Body=json.dumps(request_body),
        ContentType="application/json",
    )

    for event in response["Body"]:
        yield event["PayloadPart"]["Bytes"]

In [ ]:
# print stream response
response = invoke_endpoint_stream(endpoint_name, input)

client = sseclient.SSEClient(response)
for event in client.events():
    if event.data == "[DONE]":
        break

    data = json.loads(event.data)
    if data.get("choices"):
        print(data["choices"][0]["delta"].get("content", ""), end="")

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you can delete the endpoint and avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()